In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [16]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk
import datetime
import youtube_dl
from moviepy.editor import VideoFileClip, AudioFileClip
import urllib.request

### Testing New Uniform GUI

In [46]:
import tkinter
import tkinter.messagebox
import customtkinter

customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"


class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        self.title("CustomTkinter complex_example.py")
        self.geometry(f"{1100}x{580}")
        self.grid_columnconfigure(0, weight=0)
        self.grid_columnconfigure(1, weight=2)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        #Sidebar Frame
        self.sidebar_frame = customtkinter.CTkFrame(self, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=3, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)
        self.title_label = customtkinter.CTkLabel(self.sidebar_frame, text="Reddit Archiver", font=customtkinter.CTkFont(size=20, weight="bold"))
        self.title_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        self.footer_label = customtkinter.CTkLabel(self.sidebar_frame, text="by dandex200")
        self.footer_label.grid(row=5, column=0, padx=20, pady=(20, 10))

        #Main Frame
        self.main_frame = customtkinter.CTkFrame(self, corner_radius=0)
        self.main_frame.grid(row=0, column=1, rowspan=3, sticky="nsew")
        self.username_label = customtkinter.CTkLabel(self.main_frame, text="Username:")
        self.username_label.grid(row=0, column=1, padx=20, pady=(20, 5), sticky="w")
        self.username_entry = customtkinter.CTkTextbox(self.main_frame, width=250, height=30)
        self.username_entry.grid(row=1, column=1, padx=20, pady=(0, 20), sticky="w")
        self.output_label = customtkinter.CTkLabel(self.main_frame, text="Select Output Directory:")
        self.output_label.grid(row=2, column=1, padx=20, pady=(20, 5), sticky="w")
        self.output_entry = customtkinter.CTkTextbox(self.main_frame, width=250, height=30)
        self.output_entry.grid(row=3, column=1, padx=20, pady=(0, 20), sticky="w")

        self.checkbox_slider_frame = customtkinter.CTkFrame(self.main_frame)
        self.checkbox_slider_frame.grid(row=4, column=1, padx=(20, 20), pady=(20, 0), sticky="nsew")
        self.checkbox_1 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        self.checkbox_1.grid(row=1, column=0, pady=(20, 0), padx=20, sticky="n")
        self.checkbox_2 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        self.checkbox_2.grid(row=2, column=0, pady=(20, 0), padx=20, sticky="n")
        self.checkbox_3 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
        self.checkbox_3.grid(row=3, column=0, pady=20, padx=20, sticky="n")

        self.start_button = customtkinter.CTkButton(self.main_frame, command=self.button_pressed, text="START")
        self.start_button.grid(row=5, column=2, padx=20, pady=10)

        #Functions
    def button_pressed(self):
        print("button click")

if __name__ == "__main__":
    app = App()
    app.mainloop()

invalid command name "139945526675968<lambda>"
    while executing
"139945526675968<lambda>"
    ("after" script)
invalid command name "139945527075136<lambda>"
    while executing
"139945527075136<lambda>"
    ("after" script)


## User Input

In [2]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files


## Scraping

In [3]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

form_data = {"over18": "yes"}
response = requests.get(user_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("Over 18, bypassing...")
    age_url =soup.select_one('link[rel="canonical"]')['href']
    print(age_url)
    new_response = requests.post(age_url, data=form_data, headers=headers)
    soup = BeautifulSoup(new_response.content, 'html.parser')
    title_check = soup.title.text.split(' ')[0]
    if title_check == "overview":
        print("bypass working")
    else:
        print("bypass failed")

Over 18, bypassing...
https://www.reddit.com/over18/?dest=https%3A%2F%2Fold.reddit.com%2Fuser%2FThrowRAlostinspace0
bypass working


In [4]:
#Testing Scraping all pages of user profile and adding each to an array of soups
user_url = 'https://old.reddit.com/user/'+user_input
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
form_data = {"over18": "yes"}
soups = []
response = requests.get(user_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
    soups.append(soup) #Append first page
else:
    print("Over 18, bypassing...")
    age_url =soup.select_one('link[rel="canonical"]')['href']
    print(age_url)
    new_response = requests.post(age_url, data=form_data, headers=headers)
    soup = BeautifulSoup(new_response.content, 'html.parser')
    title_check = soup.title.text.split(' ')[0]
    if title_check == "overview":
        print("bypass working")
        soups.append(soup) #Append first page
    else:
        print("bypass failed")
while True:
    next_button = soup.find('span', class_='next-button')
    if next_button:
        href_link = next_button.find('a')['href']
        response = requests.get(href_link, data=form_data, headers=headers)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        title_check = soup2.title.text.split(' ')[0]
        if title_check == "overview":
            print("working")
            soups.append(soup2) #Append first page
            soup = soup2
        else: #TODO BELOW IS NOT WORKING
            print("Over 18, bypassing...")
            age_url =soup.select_one('link[rel="canonical"]')['href']
            print(age_url)
            new_response = requests.post(age_url, data=form_data, headers=headers)
            soup = BeautifulSoup(new_response.content, 'html.parser')
            title_check = soup.title.text.split(' ')[0]
            if title_check == "overview":
                print("bypass working")
                soups.append(soup2) #Append first page
                soup = soup2
            else:
                print("bypass failed")
    else:
        break

Over 18, bypassing...
https://www.reddit.com/over18/?dest=https%3A%2F%2Fold.reddit.com%2Fuser%2FThrowRAlostinspace0
bypass working


In [5]:
len(soups)

1

### Profile Info

In [6]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''
comment_karma = int(comment_karma.replace(',', ''))
post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''
post_karma = int(post_karma.replace(',', ''))

user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0
for soup in soups:
    for comment in soup.find_all('div', class_='comment'):
        num_comments += 1
    for post in soup.find_all('div', class_='link'):
        num_posts += 1

print("Username: " + username)
print("Comment Karma: %d"  % comment_karma)
print("Post Karma: %d" % post_karma)
print("Creation Date: " + user_creation_date)
print("Posts Count: %d" % num_posts)
print("Comments Count: %d" % num_comments)

Username: ThrowRAlostinspace0
Comment Karma: 3
Post Karma: 5
Creation Date: 2023-06-17T17:03:50+00:00
Posts Count: 2
Comments Count: 16


### Comments

In [7]:
comments = []
comment_count = 0
for soup in soups[:1]: 
    spans = []
    score = ""
    score_hidden = ""
    date_format = "%Y-%m-%dT%H:%M:%S%z"

    for comment in soup.find_all('div', class_='comment'):
        utc_date = comment.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        spans = comment.find('span', class_='score unvoted')
        if spans is not None:
            score = spans.text
        else:
            score = comment.find('span', class_='score-hidden').text
        subreddit = comment.find('a', class_="subreddit").text
        permalink = comment.find('a', class_="bylink")['href']
        text = comment.find('div', class_='md-container').text
        comment_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'text': text
        }
        comments.append(comment_data)
        comment_count += 1

    print(comments)
    print(comment_count)

[{'utc_date': '2023-07-26T17:32:29+00:00', 'score': '2 points', 'subreddit': 'SluttyConfessions', 'permalink': 'https://old.reddit.com/r/SluttyConfessions/comments/15aa292/am_i_23f_a_slut/jtjsn4m/', 'text': 'I will make sure to try not to worry about that, even my friends told me my past is pretty normal\n\n'}, {'utc_date': '2023-07-26T17:31:23+00:00', 'score': '1 point', 'subreddit': 'SluttyConfessions', 'permalink': 'https://old.reddit.com/r/SluttyConfessions/comments/15aa292/am_i_23f_a_slut/jtjsgkr/', 'text': 'Well I do love him, and I wish for him to be my last and vice versa\n\n'}, {'utc_date': '2023-07-26T17:30:41+00:00', 'score': '1 point', 'subreddit': 'SluttyConfessions', 'permalink': 'https://old.reddit.com/r/SluttyConfessions/comments/15aa292/am_i_23f_a_slut/jtjsci8/', 'text': 'Guess I just sometimes forget those are normal experiences, especially if he asks me about then, I just start feeling like I did something bad, like I shouldnt have partners before him\n\n'}, {'utc_da

### Posts

In [8]:
#Counting Posts
post_count = 0
for soup in soups: 
    for post in soup.find_all('div', class_='link'):
        post_count += 1
    print(post_count)

2


In [9]:
output_dir

'/home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files'

In [14]:
posts = []
post_count = 0

for soup in soups: 

    utc_date = 0
    post_type = ""
    subreddit = ""
    post_title = ""
    file_path = ""
    permalink = ""
    score = 0
    text = ""

    #self post
    for self_post in soup.find_all('div', class_='self'):
        post_count += 1
        utc_date = self_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "self"
        subreddit = self_post.find('a', class_="subreddit").text
        post_title = self_post.find('a', class_="title").text
        score = self_post.find('div', {'class': 'score unvoted'}).text
        permalink = self_post.find('a', class_="bylink")['href']

        if self_post.find('div', {'class': 'expando-button'}): 

            self_text_url = permalink

            headers2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response2 = requests.get(self_text_url, headers=headers2)
            soup2 = BeautifulSoup(response2.content, "html.parser")
            post_content = soup2.find("div", class_="entry")
            #md_element = post_content.find("div", class_="md-container")
            #post_text = md_element.text
            print(post_content)

            #text = post_text
            
        else:
            text = None
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'text': text
        }
        posts.append(post_data)
        
    #crosspost
    for cross_post in soup.find_all('div', {'class':'link', 'data-crosspost-root-title': True}):
        post_count += 1
        utc_date = cross_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "crosspost"
        subreddit = cross_post.find('a', class_="subreddit").text
        post_title = cross_post.find('a', class_="title").text
        score = cross_post.find('div', {'class': 'score unvoted'}).text
        permalink = cross_post.find('a', class_="bylink")['href']
        cross_post_link = cross_post.find('a', class_="title")['href']
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            "CrossPost Link": cross_post_link
        }
        posts.append(post_data)
    #link
    domains = ['old.reddit.com', 'i.redd.it', 'v.redd.it', 'i.imgur.com']
    for link_post in soup.find_all(lambda tag: tag.has_attr('data-domain') and not any(domain in tag['data-domain'] for domain in domains )and 'self' not in tag.get('class', [])):
        post_count += 1
        utc_date = link_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "link"
        subreddit = link_post.find('a', class_="subreddit").text
        post_title = link_post.find('a', class_="title").text
        score = link_post.find('div', {'class': 'score unvoted'}).text
        permalink = link_post.find('a', class_="bylink")['href']
        
        if link_post.find('div', {'class': 'expando-button'}): #If expando button present
            #Download permalink
            form_data = {"over18": "yes"}
            response = requests.get(permalink, data=form_data, headers={'User-Agent': 'Mozilla/5.0'})
            post_soup = BeautifulSoup(response.content, 'html.parser')
            #Extract text
            link = post_soup.find('a', class_="title")['href']
        else:
            link = None
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'link': link
        }
        posts.append(post_data)
    #video
    for video_post in soup.find_all('div', {'class':'link', "data-domain": "v.redd.it", "data-kind": "video"}):
        post_count += 1
        utc_date = video_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "video"
        subreddit = video_post.find('a', class_="subreddit").text
        post_title = video_post.find('a', class_="title").text
        score = video_post.find('div', {'class': 'score unvoted'}).text
        permalink = video_post.find('a', class_="bylink")['href']
        vid_link = video_post.find('a', class_="title")['href']
        
        ydl_opts = {
            'outtmpl': f'{output_dir}/%(title)s.%(ext)s',
            #'outtmpl': output_dir + ,
            'merge_output_format': 'mp4',
        }
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([vid_link])
        #Merge and Remove 
        for filename in os.listdir(output_dir):
            if filename.endswith(".mp4") and "merged" not in filename:
                video_path = os.path.join(output_dir, filename)
                audio_filename_prefix = filename[:8]
                
                for file in os.listdir(output_dir):
                    if file.startswith(audio_filename_prefix) and file.endswith(".m4a"):
                        audio_path = os.path.join(output_dir, file)
                        output_filename = os.path.splitext(filename)[0] + "_merged.mp4"
                        output_path = os.path.join(output_dir, output_filename)
                        video = VideoFileClip(video_path)
                        audio = AudioFileClip(audio_path)
                        video = video.set_audio(audio)
                        video.write_videofile(output_path, codec="libx264", audio_codec="aac")
                        break
                os.remove(video_path)
                os.remove(audio_path)
                vid_path = output_path
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'filepath': vid_path,
            'text' : text
        }
        posts.append(post_data)
    #pictures 

    for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com", "data-is-gallery": "true"}):
        post_count += 1
        utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "pic-gallery"
        subreddit = pic_post.find('a', class_="subreddit").text
        post_title = pic_post.find('a', class_="title").text
        score = pic_post.find('div', {'class': 'score unvoted'}).text
        permalink = pic_post.find('a', class_="bylink")['href']
        gallery_link = pic_post.find('a', class_="title")['href'] 
        filepaths = []
        headers3 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response3 = requests.get(gallery_link, headers=headers3)
        soup3 = BeautifulSoup(response3.content, "html.parser")
        presentation_div = soup3.find("div", {"role": "presentation"})
        a_elements = presentation_div.find_all("a")
        if a_elements:
            for a_element in a_elements:
                href_link = a_element.get("href")
                if href_link and href_link.startswith("https://preview"):
                    pic_filename = href_link.split("/")[-1]  
                    pic_filename = pic_filename.split(".jpg")[0] + ".jpg"  
                    pic_path = os.path.join(output_dir, pic_filename)
                    urllib.request.urlretrieve(href_link, pic_path)
                    filepaths.append(pic_path)
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'filepath': filepaths
        }
        posts.append(post_data)

    for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com", "data-is-gallery": "false"}):
        post_count += 1
        utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "pic-old"
        subreddit = pic_post.find('a', class_="subreddit").text
        post_title = pic_post.find('a', class_="title").text
        score = pic_post.find('div', {'class': 'score unvoted'}).text
        permalink = pic_post.find('a', class_="bylink")['href']
        pic_link = pic_post.find('a', class_="title")['href']
        pic_path = ""
        pic_filename = pic_link.split("/")[-1]  
        pic_path = os.path.join(output_dir, pic_filename)
        urllib.request.urlretrieve(pic_link, pic_path)
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'filepath': pic_path
        }
        posts.append(post_data)
    for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.redd.it"}):
        post_count += 1
        utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "pic-i"
        subreddit = pic_post.find('a', class_="subreddit").text
        post_title = pic_post.find('a', class_="title").text
        score = pic_post.find('div', {'class': 'score unvoted'}).text
        permalink = pic_post.find('a', class_="bylink")['href']
        #pic_link = pic_post.find('a', class_="title")['href']
        pic_link = pic_post['data-url']
        print(pic_link)
        pic_path = ""
        pic_filename = pic_link.split("/")[-1]  
        pic_path = os.path.join(output_dir, pic_filename)
 
        urllib.request.urlretrieve(pic_link, pic_path)
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type,
            'filepath': pic_path
        }
        posts.append(post_data)

    for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.imgur.com"}):
        post_count += 1
        utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
        utc_date = datetime.datetime.strptime(utc_date, date_format)
        utc_date = utc_date.isoformat()
        post_type = "pic-imgur"
        subreddit = pic_post.find('a', class_="subreddit").text
        post_title = pic_post.find('a', class_="title").text
        score = pic_post.find('div', {'class': 'score unvoted'}).text
        permalink = pic_post.find('a', class_="bylink")['href']
        post_data = {
            'utc_date': utc_date,
            'score': score,
            'subreddit': subreddit,
            'permalink': permalink,
            'title': post_title,
            'type': post_type
        }
        posts.append(post_data)

    print(post_count)

None
None
2


In [15]:
#Test if 25 Posts identified (Number of each page)
if (post_count + comment_count) != (num_comments+num_posts):
    print("Error")
else:
    
    print("working")

Error


In [16]:
num_posts = len(posts)
num_comments = len(comments)

In [17]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [18]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

In [19]:
with open('Temp_Files/user_data.json') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "user_info": {
        "username": "ThrowRAlostinspace0",
        "user_creation_date": "2023-06-17T17:03:50+00:00",
        "num_posts": 2,
        "num_comments": 16,
        "karma": {
            "post_karma": 5,
            "comment_karma": 3
        }
    },
    "posts": [
        {
            "utc_date": "2023-07-27T14:29:11+00:00",
            "score": "14",
            "subreddit": "r/AskRedditNSFW",
            "permalink": "https://old.reddit.com/r/AskRedditNSFW/comments/15b3dp1/am_i_23f_a_slut/",
            "title": "Am I (23F) a slut?",
            "type": "self",
            "text": ""
        },
        {
            "utc_date": "2023-07-26T16:13:11+00:00",
            "score": "6",
            "subreddit": "r/SluttyConfessions",
            "permalink": "https://old.reddit.com/r/SluttyConfessions/comments/15aa292/am_i_23f_a_slut/",
            "title": "Am I (23F) a slut?",
            "type": "self",
            "text": ""
        }
    ],
    "comments": [
   

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* blank

**Minor Additions**
* Accounting for pinned posts
* Profile pictures (snoovatar??)
* Trophies
* Add over 18 bypass for old.reddit

In [15]:
#Clear Temp_Files
for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

2
2
2
